This notebook is set up to test code as needed.

In [ ]:
# import packages
import pandas as pd
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import download_data
import load_data
from column_checks import get_dtypes
from filepaths import *
import impute_hourly_profiles
import data_cleaning
import output_data
import emissions
import validation
import gross_to_net_generation
import eia930

year = 2020
path_prefix = f"{year}/"

In [ ]:
# load the raw and cleaned eia930 data to compare
raw_930_file = outputs_folder(f"{path_prefix}/eia930/eia930_raw.csv")
clean_930_file = outputs_folder(f"{path_prefix}/eia930/eia930_elec.csv")

eia930_raw = eia930.load_chalendar_for_pipeline(raw_930_file, year=year)
eia930_data = eia930.load_chalendar_for_pipeline(clean_930_file, year=year)

eia930_merged = eia930_raw.merge(eia930_data, how="left", on=["ba_code","fuel_category_eia930","datetime_utc","datetime_local","report_date"], suffixes=("_raw","_cleaned"))

In [ ]:
# calculate how well correlated the raw and cleaned data is
correlations = eia930_merged.groupby(["ba_code","fuel_category_eia930","report_date"], dropna=False)[["net_generation_mwh_930_raw","net_generation_mwh_930_cleaned"]].corr().reset_index()
correlations = correlations[correlations["level_3"] == "net_generation_mwh_930_raw"]
correlations = correlations.drop(columns=["level_3","net_generation_mwh_930_raw"])
correlations = correlations.rename(columns={"net_generation_mwh_930_cleaned":"correlation_with_raw"})
correlations = correlations[correlations["report_date"].dt.year == 2020]
correlations

In [ ]:
ba = "PJM"
fuel = "coal"

correlations[(correlations["ba_code"] == ba) & (correlations["fuel_category_eia930"] == fuel)]

In [ ]:
correlations[correlations["correlation_with_raw"] < 0.1]

In [ ]:
ba = "BPAT"
fuel = "nuclear"

data_to_plot = eia930_merged[(eia930_merged["ba_code"] == ba) & (eia930_merged["fuel_category_eia930"] == fuel)]

px.line(data_to_plot, x="datetime_local", y=["net_generation_mwh_930_raw","net_generation_mwh_930_cleaned"])